In [26]:
from ipywidgets import interact
from numpy.random import normal, seed, uniform
from matplotlib import pyplot as plt
from functools import partial
import torch

seed(42)

def plot_function(f, tx=None, ty=None, title=None, min=-2, max=2, figsize=(6,4)):
    x = torch.linspace(min, max, steps=100)
    fig,ax = plt.subplots(figsize=figsize)
    ax.plot(x,f(x))
    if tx is not None: ax.set_xlabel(tx)
    if ty is not None: ax.set_ylabel(ty)
    if title is not None: ax.set_title(title)
    return ax

def noise(x, scale):
    return normal(scale=scale, size=x.shape)

def add_noise(x, mult, add):
    return x * (1 + noise(x, mult)) + noise(x, add)

def quad(a, b, c, x):
    return a*x**2 + b*x + c

def mk_quad(a, b, c):
    return partial(quad, a, b, c)

In [37]:
import math

def mse(pred,y):
    return ((pred-y)**2).mean()

def rmse(pred,y):
    return round(math.sqrt(mse(pred, y), 6))

def quad_mse(params):
    f = mk_quad(*params)
    return mse(f(x), y)

def quad_rmse(params):
    f = mk_quad(*params)
    return rmse(f(x), y)

In [38]:
f = mk_quad(3, 2, 1)
x = torch.linspace(-2, 2, steps=20)[:,None]
y = add_noise(f(x), 0.3, 1.5)

@interact(a=1.5, b=1.5, c=1.5)
def plot_quad(a, b, c):
    f_guess = mk_quad(a, b, c)
    mse_loss = mse(f_guess(x), y)
    rmse_loss = r_mse(f_guess(x), y)

    ax = plot_function(
        f_guess,
        min=-3,
        max=3,
        title=f"MSE: {mse_loss:.2f}, RMSE: {rmse_loss:.2f}"
    )
    ax.scatter(x, y)

interactive(children=(FloatSlider(value=1.5, description='a', max=4.5, min=-1.5), FloatSlider(value=1.5, descr…

In [101]:
abc = torch.tensor([1.5, 1.5, 1.5], requires_grad=True)
loss = quad_mse(abc)
loss.backward()

# grad represents the direction you should go.
# positions correspond to parameters
# negative values means you should increase the parameter
# higher absolute values indicate greater relative importance
abc.grad

tensor([-8.8910, -4.1143, -2.8729])

In [116]:
# basically every machine learning model does this repeatedly
# This is gradient descent, bitches
with torch.no_grad():
    abc -= abc.grad * 0.01
    loss = quad_mse(abc)

print(f'loss={loss:.2f}; abc={abc}')


loss=6.55; abc=tensor([2.7447, 2.0760, 1.9022], requires_grad=True)


In [122]:
# Ok, so this works fine for quadratic equations.
# How do we make this work for much more complicated
# things like image identification?

def rectified_linear(m, b, x):
    y = m*x + b
    # clip will force all negative values to match the 2nd param (0.0)
    return torch.clip(y, 0.0)

def dub_rectified_linear(m1, b1, m2, b2, x):
    return rectified_linear(m1, b1, x) + rectified_linear(m2, b2, x)

# essentially, just keep adding linear functions
# together to represent arbitrarily complex functions.

In [121]:
@interact(m=1.5, b=1.5)
def plot_rect_lin(m, b):
    func = partial(rectified_linear, m, b)
    ax = plot_function(
        func,
        min=-3,
        max=3,
    )

interactive(children=(FloatSlider(value=1.5, description='m', max=4.5, min=-1.5), FloatSlider(value=1.5, descr…

In [124]:
@interact(m1=-1.5, b1=-1.5, m2=1.5, b2=1.5)
def plot_double_rect_lin(m1, b1, m2, b2):
    func = partial(dub_rectified_linear, m1, b1, m2, b2)
    ax = plot_function(
        func,
        min=-3,
        max=3,
    )

interactive(children=(FloatSlider(value=-1.5, description='m1', max=1.5, min=-4.5), FloatSlider(value=-1.5, de…